1. [**Введение**](#start)
  * [Обзор данных](#review)
2. [**Предобработка данных**](#preclean)
  * [Дубликаты и пропуски](#duplicated)
  * [Аномальные значения](#what)
3. [**Анализ рынка заведений общественного питания**](#restaurant)
  * [Количество заведений по категориям](#category)
  * [Количество посадочных мест по категориям](#seats)
  * [Количество сетевых заведений](#chain)
  * [Соотношение сетевых и несетевых заведений](#chain_perc)
  * [Сети с самым большим количеством точек](#biggest_net)
  * [Категории заведений по районам](#district)
  * [Средние рейтинги у разных категорий общепита](#rating") 
  * [Распределение стреднего рейтинга по районам](#meanrating)
  * [Распределение заведений](#cafecluster)
  * [Топ-15 улиц по количеству заведений](#top) 
  * [Распределение среднего чека по районам](#avg) 
  * [Зависимость среднего чека](#avgcorr) 
  * [Ценовая категория заведений](#price) 
  * [Вывод](#so) 
4. [**Анализ кофейного рынка**](#coffee)
  * [Количество круглосуточных кофеен](#24/7) 
  * [Распределения рейтинга кофеен по районам](#cof_rat) 
  * [Средняя стоимость чашки кофе и средний чек по районам](#avgcup) 
  * [Количество сетевых кофеен](#coffeenet) 
  * [Средняя стоимость чашки кофе и средний чек в сетевых и несетевых кофейнях](#netavg) 
  * [Распределение среднего чека кофеен по районам](#coffeedistrict) 
  * [Кофейни с самым высоким средним чеком](#coffeetop) 
  * [Соотношение посадочных мест](#coffeeseats) 
  * [Вывод](#soo) 
5. [**Общий вывод**](#conclution)
6. [**Рекомендации**](#recommendations)

**1.Введение** <a id="start"></a> 

Исходные данные: датасет, основанный на данных Яндекс.Карт и Яндекс.Бизнеса.

Задачи исследования: проанализировать рынок заведений общественного питания в целом и кофеен в частности, выявить особенности и рекомендации.

**1.1.Обзор данных** <a id="review"></a> 

In [1]:
#импортируем все необходимые для работы библиотеки
import pandas as pd
import seaborn as sns
sns.set_style('whitegrid') #задаем стиль для графиков
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight') #задаем стиль для графиков
plt.rcParams['figure.figsize'] = [15, 10] #и их размер
import plotly.express as px
from plotly import graph_objects as go
import warnings
import json
import folium


warnings.filterwarnings('ignore') #игнорируем ошибки
pd.set_option('display.float_format', '{:.2f}'.format) #формат числел с плавающей точкой с двумя знаками после знака

In [5]:
path = '//datasets//'
data = pd.read_csv(path + 'moscow_places.csv')

data.info()

FileNotFoundError: [Errno 2] No such file or directory: '//datasets//moscow_places.csv'

В таблице представлены данные о названиях заведений, их категориях, адресах (включая районы), часы работы, географические координаты, рейтинг, средний чек, является ли заведение сетевым и количество посадочных мест.

**2.Предобработка данных** <a id="preclean"></a> 

***2.1.Дубликаты и пропуски*** <a id="duplicated"></a> 

In [ ]:
#начнем с пропусков
def nan_barh(data):
        (
            (data.isna().mean()*100)
            .to_frame()
            .rename(columns = {0:'null'})
            .query('null > 0')
            .sort_values(by = 'null', ascending = True)
            .plot(kind = 'barh', figsize = (19,6))   
        );
        
nan_barh(data)

Больше всего пропусков в столбце со средней стоимостью чашки кофе, что можно объяснить тем, что далеко не во всех заведениях есть кофейные напитки. Пропуски в остальных полях можно объяснить тем, что данные в дадасете из открытых источников или заполняются пользователем, соответственно их могли просто не указать.

Пропуски заполнять не будем, оставим как есть.

In [ ]:
#проверяем наличие двух одинаковых заведений
data.duplicated(['name', 'address']).sum()

In [ ]:
#проверяем категории
data['category'].unique()

In [ ]:
#проверяем названия округов
data['district'].unique()

In [ ]:
#приведем все к нижнему регистру
data['name'] = data['name'].str.lower()
data['address'] = data['address'].str.lower()

In [ ]:
data[data.duplicated(subset=['name', 'address'])]

В данных эти строки помечаются как дубликаты, хотя названия и адреса разные.

In [ ]:
data['name']=data['name'].replace({'9 bar coffe':'9 bar coffee', 'arzon':'arzon food','bổ': 'bổ',
                                'cafe 13':'cafe13','caffeterria':'caffetteria','cantina city free flow cafe':'cantina city',
                                 'cofe fest':'cofefest','coffee in':'coffee inn','corner cafe & kitchen':'corner cafe&kitchen',
                                  'dormouse coffee shop':'dormouse', 'drive café':'drive cafe',
                                  'fast шашлык':'fastshashlik','festa & тесто':'festa&тесто',
                                  'free & со':'free&co','hanoi food&drinks':'hanoi',
                                  'i-chef бистро':'i-chef','iq кофе':'iq coffee',
                                  "it's сот - кофейня":"it's сот-кофейня",
                                  'jeffrey’s coffeeshop':"jeffrey's coffeeshop",'k-town korean bbq':'k-town',
                                  'lavaш':'lavash', 'lucky eye. ближневосточной lounge bar':'lucky eye',
                                  'main food':'mainfood', 'meat лав':'meatлав', 'яндекс лавка':'яндекс.лавка',
                                  'шашлычный двор № 1':'шашлычный дворик № 1','шашлычный двор':'шашлычный дворик',
                                  'шашлычный дворик':'шашлычный дворик № 1','шашлыкоff':'шашлыкофф','шашлык house':'шашлык хаус',
                                  'чудо печка':'чудо печка','чайхона № 1':'чайхона №1','чайхана-халва':'чайхона халва',
                                  'чайхона':'чайхана','хлебо mania':'хлебомания','хлеба&зрелищ пиццерия':'хлеба&зрелищ',
                                  'хинкали-gали!':'хинкали-gали','хинкали gали!':'хинкали gали','хинкали - gали!':'хинкали-gали',
                                  'хинкали-gали':'хинкали gали','топ chick':'топ чик', 'тбилисо, трактир':'тбилисо'})
data['name'] = data['name'].replace('ё', 'е')

In [ ]:
data.name.nunique()

В данных огромное количество неявных дубликатов в столбце с названиями, что можно объяснить ручным вводом данных.

***2.2.Аномальные значения*** <a id="what"></a> 

In [3]:
#смотрим есть ли аномалии в рейтингах
data['rating'].describe()

sns.boxplot(data['rating']);

NameError: name 'data' is not defined

Редкими значениями можно назвать все, что ниже рейтинга в 3.7 и выше 4.7, однако это все еще реалистичные цифры. Основные оценки сосредоточены между 4 и 4,5

In [ ]:
data[['seats', 'middle_avg_bill']].apply(['count', 'median', 'mean', 'min', 'max'])

In [ ]:
sns.violinplot(data['seats']);
plt.show()

data[data['seats'] > 600].count()

Есть заведения с нереалистично большим количеством посадочных мест, которые выглядят как ошибки в данных. Оставим только заведения, в которых посадочных мест меньше 400 (включая максимальную границу)

In [ ]:
data[data['seats']>600]

In [ ]:
df = data.query('seats.isna() | seats<=600')
df.info()

In [ ]:
df[df['middle_avg_bill'] == 35000]

Это заведение выглядит странно. Данные на Яндекс.Картах такие же, значит ошибки при выгрузке не было, однако на других сайтах указано, что в этом здании находится онкологический научный центр. Не понятно...

*Вывод*

В данных обнаружились аномалии в количестве посадочных мест, и среднем чеке (в одном очень странном заведении).
Есть большое количество пропусков, но они объясняются ручным заполнением информации об организациях.

*Пополнение таблицы*

Пополним датасет столбцами с названием улиц и является ли заведение круглосуточным.

In [ ]:
#приводим к строковому формату и разбиваем по запятой
df['address'] = df['address'].astype('str')
df['address'] = df['address'].str.split(', ')

In [ ]:
#добавление колонки с улицей
def address(row):
    street = row['address']
    return street[1]
    
df['street'] = df.apply(address, axis=1)

df['street'] = df['street'].str.strip()

In [ ]:
#приводим к строковому формату и разбиваем по запятой
df['hours'] = df['hours'].astype('str')
df['hours'] = df['hours'].str.split(', ')

In [ ]:
#добавление колонки с круглосуточными заведениями
def daybyday(row):
    time = row['hours']
    if ('круглосуточно' in time)&('ежедневно' in time):
        return True
    else:
        return False
        
df['is_24/7'] = df.apply(daybyday, axis = 1)
df.head(20)

**3.Анализ рынка заведений общественного питания** <a id="restaurant"></a> 

***3.1.Количество заведений по категориям*** <a id="category"></a> 

In [ ]:
df.pivot_table(index='category', values='name', aggfunc='count').sort_values(by='name',ascending=False).plot(kind='bar', 
                                                                    title='Распределение количества заведений по категориям',
                                                                    xlabel='Категория заведений',
                                                                    ylabel='Количество');

Больше всего заведений в категориях рестораны и кафе, на третьем месте расположились кофейни, меньше всего столовых и булочных.

***3.2.Количество посадочных мест по категориям*** <a id="seats"></a> 

In [ ]:
sns.boxplot(data=df, x='seats', y='category');
plt.title('Количество посадочных мест по категориям');
plt.xlabel('Посадочные места');
plt.ylabel('Категория заведения');
plt.show()

df.groupby('category')['seats'].agg(['median', 'mean']).sort_values(by='mean', ascending=False)

В среднем, больше всего посадочных мест в ресторанах, барах и кофейнях, меньше всего в булочных.

***3.3.Количество сетевых заведений*** <a id="chain"></a> 

In [ ]:
net = df.groupby('chain')['category'].agg('count')

fig = go.Figure(data=[go.Pie(labels=['Не сеть', 'Сеть'], values=net)])
fig.update_layout(title='Количество сетевых заведений')
fig.show()

Сетевых заведений из общего количества всего 38%

***3.4.Соотношение сетевых и несетевых заведений*** <a id="chain_perc"></a> 

In [ ]:
chain = df.groupby('category')[['name', 'chain']].agg({'chain':'sum',
                                                        'name':'count'})
chain.columns=['chain', 'total']
chain['perc'] = chain['chain']/chain['total']
chain = chain.sort_values(by='perc')
fig = px.bar(chain, y='perc', title='Соотношение сетевых заведений')
fig.show()

Самый большой процент сетевых  булочных, пиццерий и кофеен, меньше всего баров, заведения быстрого питания только на 4 месте!

***3.5.Сети с самым большим количеством точек*** <a id="biggest_net"></a> 

In [ ]:
top_net = df.query('chain == 1')
top_net.pivot_table(index='name', values='address', aggfunc='count').sort_values(by='address',
                                                                                 ascending=False).head(15).plot(kind='barh');

Почти все эти заведения являются франшизой и имеет распространение по всей России или просто большое количество точек по всей Москве (тоже франшизных)

***3.6.Категории заведений по районам*** <a id="district"></a> 

In [ ]:
district = df.pivot_table(index=('district'), 
                            values='name', columns='category', aggfunc='count')
px.bar(district, title='Распределение категорий заведений по районам').update_layout(width=1000,height=800,
                                                                                    xaxis_title='Район',
                                                                                    yaxis_title='Количество заведений')

Самое большое количество заведений расположено в Центральном автономном округе, меньше всего заведений находятся в Северо-Западном автономном округе.

***3.7.Средние рейтинги у разных категорий общепита*** <a id="rating"></a> 

In [ ]:
plt.figure(figsize=(15, 5))
plt.title('Рейтинг по категориям');
sns.barplot(data=df, x='category', y='rating');

В среднем рейтинги у разных категорий заведений отличается не сильно, однако можно заметить, что самый низкий средний рейтинг у заведений фастфуда. Это может быть связано с тем, что скорость обслуживания достаточно быстрая, а вот качество не всегда отвечает потребностям потребителей.

***3.8.Распределение стреднего рейтинга по районам*** <a id="meanrating"></a> 

In [ ]:
rating_df = df.groupby('district', as_index=False)['rating'].agg('mean')
rating_df

In [ ]:
from folium import Map, Choropleth

state_geo = '/datasets/admin_level_geomap.geojson'
moscow_lat, moscow_lng = 55.751244, 37.618423

rating_map = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

Choropleth(
    geo_data=state_geo,
    data=rating_df,
    columns=['district', 'rating'],
    key_on='feature.name',
    fill_color='OrRd',
    fill_opacity=0.7,
    legend_name='Средний рейтинг заведений по районам',
).add_to(rating_map)

rating_map

***3.9.Распределение заведений*** <a id="cafecluster"></a> 

In [ ]:
from folium import Marker, Map
from folium.plugins import MarkerCluster

moscow_lat, moscow_lng = 55.751244, 37.618423
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
marker_cluster = MarkerCluster().add_to(m)

def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)
    
df.apply(create_clusters, axis=1)

m

Одинокое заведение в торговом центре "Мирус авто"

***3.10.Топ-15 улиц по количеству заведений*** <a id="top"></a> 

In [ ]:
street = df.pivot_table(index='street', values='name', columns='category', aggfunc='count')

#сделаем отдельный столбец тотал, чтобы сортировать по нему
street.columns=['bar', 'bread', 'fastfood', 'cafe', 'coffeeshop', 'pizza', 'restaurant', 'eatery']
street['total'] = street['bar']+street['bread']+ \
street['fastfood']+street['cafe']+street['coffeeshop']+ \
street['pizza']+street['restaurant']+street['eatery']
top_street = street.sort_values(by='total', ascending=False).head(15)

px.bar(top_street, title='Топ улиц по количеству заведений').update_layout(xaxis_title='Улица',
                                                                          yaxis_title='Количество заведений')

Больше всего заведений находится на Проспекте Мира, там преимущественно расположены кафе, рестораны и кофейни. Большое количество заведений может быть обосновано тем, что эта улица находится в Центральном автономном округе, в котором самое большое количество заведений по Москве в принципе. 

На этих улицах находится около 12,5% всех заведений Москвы.

In [ ]:
street[street['cafe']==1].head(20)

Можно предположить, что на этих улицах такое маленькое количество заведений из-за:
    
    1. Большой удаленности от станции метро 
    2. Удаленности от центра
    3. Расположение в жилых районах
    4. Отсутствие спроса на заведения общественного питания в данных районах

***3.11.Распределение среднего чека по районам*** <a id="avg"></a> 

In [ ]:
avg_bill_data = df.groupby('district', as_index=False)['middle_avg_bill'].agg('mean')
avg_bill_data

avg_bill_data_plot = df.groupby('district')['middle_avg_bill'].agg('mean').sort_values()
px.bar(avg_bill_data_plot, title='Средний чек по районам').update_layout(xaxis_title='Район',
                                                                        yaxis_title='Средний чек')

In [ ]:
from folium import Map, Choropleth

state_geo = '/datasets/admin_level_geomap.geojson'
moscow_lat, moscow_lng = 55.751244, 37.618423

avg_map = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

Choropleth(
    geo_data=state_geo,
    data=avg_bill_data,
    columns=['district', 'middle_avg_bill'],
    key_on='feature.name',
    fill_color='OrRd',
    fill_opacity=0.7,
    legend_name='Средний чек по районам',
).add_to(avg_map)

avg_map

На средний чек влияет не только удаленность от центра, но и общий престиж округа, его инфраструктура и расположение известных объектов(парки, институты и университеты).
Чем более туристически привлекательный район, тем выше средний чек.

***3.12 Зависимость среднего чека*** <a id="avgcorr"></a> 

In [ ]:
sns.scatterplot(data = df, x='seats', y='middle_avg_bill', hue='chain');
plt.title('Зависимость среднего чека от количества посадочных мест')
plt.xlabel('Количество посадочных мест')
plt.ylabel('Средний чек')
df['seats'].corr(df['middle_avg_bill'])

От количества посадочных мест средний чек не изменяется, основная часть колеблется в пределах 5000 рублей.

In [ ]:
sns.scatterplot(data=df, x='middle_avg_bill', y='rating', hue='chain');
plt.title('Зависимость среднего чека от рейтинга')
plt.xlabel('Средний чек')
plt.ylabel('Рейтинг')
data['rating'].corr(data['middle_avg_bill'])

Можно заметить, что средний чек у заведений с рейтингом ниже 3.5 в основном ниже, чем у заведений с рейтингом выше 4, однако корреляция не такая существенная.

In [ ]:
sns.boxplot(data=data, x='middle_avg_bill', y='category');
plt.title('Средний чек по категориям заведений')
plt.xlabel('Категория');
plt.ylabel('Средний чек');

data.pivot_table(index='category', values='middle_avg_bill', aggfunc=('mean', 'median')).sort_values(by='median',
                                                                                                     ascending=False)

Самый большой средний чек у ресторанов, после них идут бары (что ожидаемо, поскольку средний чек в барах в основном идет не только от цены позиции, но и от количества позиций в чеке). Самый низкий средний чек у столовых, что тоже объяснимо их ценовой политикой.

***3.13 Ценовая категория заведений*** <a id="price"></a> 

In [ ]:
price = data.pivot_table(index='price', values='name', columns='category', aggfunc='count')
px.bar(price, title='Ценовая категория заведений').update_layout(xaxis_title=('Ценовая категория'),
                                                                yaxis_title=('Количество заведений'))

Можно отметить, что больше всего кафе, ресторанов и кофеен средней ценовой категории, и в целом заведений средней ценовой категории больше, чем всех остальных вместе взятых. Основную часть заведений с низкой ценовой категорией занимают кофейни (скорее всего это сети типа Cofix и One Price Coffee, которые поддерживают достаточно низкую ценовую политику).

Основную часть заведений с высокими ценами занимают рестораны и бары. В целом, этот параметр тоже может объяснить их самый высокий средний чек.

***Вывод*** <a id="so"></a> 

Всего в Москве представлено больше 8000 заведений общественного питания, большую часть из которых составляют кафе, рестораны и кофейни. Из общего количества заведений 38,1% составляют сетевые. Самое большое количество посадочных мест в ресторанах и барах, самое низкое в булочных, по большей части потому, что они рассчитаны на быструю покупку с собой.

У сетевых заведений самое большое количество точек у тех, кто открывается по франшизе(или имеет франшизу), сильно распространен по Москве или по всей России. Больше всего соотношение сетевых заведений у булочных, пиццерий и кофеен.

Самое большое количество заведений находится в Центральном автономном округе, самое низкое в Северо-Западном автономном округе. На средний чек влияет не только расположение районов, но и его туристическая привлекательность, наличие известных локаций(университетов, парков и т.д), достопримечательностей. По категориям средний чек самый большой у ресторанов и баров, так как большая часть этих заведений находится в высокой ценовой категории, а также в округе с самым высоким средним чеком (ЦАО).

Средний рейтинг колеблется в пределах от 3.9 до 4.5, ниже этих значений показатель встречаются достаточно редко. Самый высокий средний рейтинг так же у Центрального автономного округа.

12,5% всех заведений находятся всего на 15 улицах, самое маленькое количество заведений находится на улицах с большой удаленностью от метро и от центра города в не самых туристических местах.

Можно сказать, что рынок общепита в Москве достаточно развит.

**Анализ кофейного рынка** <a id="coffee"></a> 

In [ ]:
#отдельная таблица для того, чтобы отделить кофейни
coffeeshop = df[df['category']=='кофейня']
print('Всего кофеен в датасете:', len(coffeeshop))

***4.1 Количество круглосуточных кофеен*** <a id="24/7"></a> 

In [ ]:
is24 = coffeeshop.groupby('is_24/7')['name'].agg('count')
is24
go.Figure(data=[go.Pie(labels=['не круглосуточно', 'круглосуточно'], values=is24)]) \
.update_layout(title='Количество круглосуточных кофеен')

Количество круглосуточных кофеен совсем небольшое.

***4.2. Распределения рейтинга кофеен по районам*** <a id="cof_rat"></a> 

In [ ]:
coffee_rating = coffeeshop.groupby('district', as_index=False)['rating'].agg('mean')
coffee_rating

In [ ]:
rating = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

Choropleth(
    geo_data=state_geo,
    data=coffee_rating,
    columns=['district', 'rating'],
    key_on='feature.name',
    fill_color='OrRd',
    fill_opacity=0.7,
    legend_name='Средний рейтинг кофеен по районам',
).add_to(rating)

rating

В среднем кофейни во всех районах имеют примерно одинаковый рейтинг.

***4.3.Средняя стоимость чашки кофе и средний чек по районам*** <a id="avgcup"></a> 

In [ ]:
sns.boxplot(x='middle_coffee_cup', y='district', data=coffeeshop);
plt.title('Средняя стоимость чашки кофе по районам');
plt.xlabel('Средняя стоимость чашки кофе');
plt.ylabel('Район');

coffeeshop.groupby('district')['middle_coffee_cup'].agg(['mean', 'median']).sort_values(by='mean')

Средняя стоимость чашки кофе самая высокая в Центральном административном округе и в Западном административном округе, самая низкая в Юго-Восточном административном округе.

In [ ]:
sns.boxplot(x='middle_avg_bill', y='district', data=coffeeshop);
plt.title('Средний чек кофеен по районам');
plt.xlabel('Средний чек');
plt.ylabel('Район');

coffeeshop.groupby('district')['middle_avg_bill'].agg(['mean', 'median']).sort_values(by='mean')

Здесь ситуация такая же, как со средней стоимостью чашки кофе, самый большой средний цек в ЦАО и ЗАО, самый низкий в ЮВАО.

***4.4.Количество сетевых кофеен*** <a id="coffeenet"></a> 

In [ ]:
isnet = coffeeshop.groupby('chain')['name'].agg('count')

go.Figure(data=[go.Pie(labels=['сеть', 'не сеть'], values=isnet)]).update_layout(title='Количество сетевых кофеен')

Распределение идет примерно 50/50. Посмотрим на другие параметры сетевых и не сетевых заведений.

***4.5.Средняя стоимость чашки кофе и средний чек в сетевых и несетевых кофейнях*** <a id="netavg"></a> 

In [ ]:
coffee_avg_cup = coffeeshop.pivot_table(index='chain', values=('middle_coffee_cup'), aggfunc='mean')
px.bar(coffee_avg_cup, title='Средняя стоимость чашки кофе').update_layout(xaxis_title='Является ли сетевым',
                                                          yaxis_title='Стоимость чашки кофе')

In [ ]:
coffee_avg_bill = coffeeshop.pivot_table(index='chain', values=('middle_avg_bill'), aggfunc='mean')
px.bar(coffee_avg_bill, title='Средний чек').update_layout(xaxis_title='Является ли сетевым',
                                                          yaxis_title='Средний чек')

Средняя стоимость чашки кофе выше в несетевых заведениях, но разница совсем несущественная, а вот средний чек выше в сетевых заведениях примерно на 60%.

***4.6.Распределение среднего чека кофеен по районам*** <a id="coffeedistrict"></a> 

In [ ]:
district_avg = coffeeshop.groupby('district')['middle_avg_bill'].agg('mean')
district_avg

In [ ]:
district = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

Choropleth(
    geo_data=state_geo,
    data=district_avg,
    columns=['district', 'rating'],
    key_on='feature.name',
    fill_color='OrRd',
    fill_opacity=0.7,
    legend_name='Средний рейтинг кофеен по районам',
).add_to(district)

district

В целом, распределение среднего чека такое же как у общепита в общем, самый высокий средний чек в Центральном автономном округе и Западном автономном округе.

***4.7. Кофейни с самым высоким средним чеком*** <a id="coffeetop"></a> 

In [ ]:
top_avg = coffeeshop.pivot_table(index='name', values='middle_avg_bill', aggfunc='mean') \
.sort_values('middle_avg_bill', ascending=False).head(10)
px.bar(top_avg).update_layout(title='Кофейни с самым высоким средним чеком',
                             xaxis_title=('Название'),
                             yaxis_title=('Средний чек'))

In [ ]:
coffeeshop[coffeeshop['name']=='кофемания']

Кофейня с самым высоким средним чеком сетевая, с высокой ценовой категорией, с достаточно хорошим рейтингом и средним счетом 1500-2000, с достаточно большим количеством посадочных мест. Также в этой кофейне представлены десерты и легкая кухня.

***4.8.Соотношение посадочных мест*** <a id="coffeeseats"></a> 

In [ ]:
sns.boxplot(data = coffeeshop, x='seats', y='district');
plt.title('Распределение количества посадочных мест по районам');
plt.xlabel('Количество посадочных мест')
plt.ylabel('Район');

In [ ]:
sns.boxplot(data=coffeeshop, x='chain', y='seats');
plt.title('Посадочные места у сетевых и несетевых заведений');
plt.xlabel('Является ли сетевой');
plt.ylabel('Количество посадочных мест');

Среднее количество посадочных мест составляет примерно от 50 до 100, в среднем у несетевых заведений количество посадочных мест меньше, чем у сетевых.

***Вывод*** <a id="soo"></a> 

В датасете представлены данные о более чем 1300 кофейнях. Почти 50% из них являются сетевыми заведениями, меньше 5% из них являются круглосуточными.

Средний рейтинг колеблется в районе 4.3, в зависимости от района меняется не сильно.

Средняя стоимость чашки кофе выше в несетевых заведениях, хотя разница не очень большая, а вот средний чек намного выше в сетевых заведениях.

Распределение среднего чека такое же, как у заведений общепита в принципе, самый большой средний чек в Центральном административном округе, самый низкий в Юго-Восточном автономном округе.

Самый большой средний чек у кофеен составляет 1860 рублей.

Количество посадочных мест в среднем составляет от 50 до 100, у сетевых заведений в среднем количество посадочных мест больше.

**Общий вывод** <a id="conclution"></a> 

  Было проведено исследование рынка общественного питание в Москве. Исходный датасет основан на данных из Яндекс.Карт и Яндекс.Бизнеса за 2022 год. Представлены такие данные как: название заведения, его адрес и географическое расположение, время работы, средний чек, ценовая категория заведения, категория заведения, данные о том, является ли заведение сетевым или нет.

  В данных обнаружились пропуски в столбцах со средней стоимостью чашки кофе, средним чеком, ценовой категорией и количеством посадочных мест. Это можно объяснить тем, что данные о заведениях заполняются польхователями или из открытых источников, а пропуске в столбце со средней стоимостью кофе может объясняться просто отсутствием кофе в заведении.
  Также были обнаружены аномалии в столбце с количеством посадочных мест. В исследовании использовались только данные о тех заведениях, в которых количество посадочных мест было меньше 400 включая эту цифру.
  Помимо этого, в данных очень большое количество неявных дубликатов в столбце с названием заведение, что также объясняется ручным вводом данных.

Всего в Москве представлено больше 8000 заведений общественного питания, большую часть из которых составляют кафе, рестораны и кофейни. Из общего количества заведений 38,1% составляют сетевые. Самое большое количество посадочных мест в ресторанах и барах, самое низкое в булочных, по большей части потому, что они рассчитаны на быструю покупку с собой.

У сетевых заведений самое большое количество точек у тех, кто открывается по франшизе(или имеет франшизу), сильно распространен по Москве или по всей России. Больше всего соотношение сетевых заведений у булочных, пиццерий и кофеен.

Самое большое количество заведений находится в Центральном автономном округе, самое низкое в Северо-Западном автономном округе. На средний чек влияет не только расположение районов, но и его туристическая привлекательность, наличие известных локаций(университетов, парков и т.д), достопримечательностей. По категориям средний чек самый большой у ресторанов и баров, так как большая часть этих заведений находится в высокой ценовой категории, а также в округе с самым высоким средним чеком (ЦАО).

Средний рейтинг колеблется в пределах от 3.9 до 4.5, ниже этих значений показатель встречаются достаточно редко. Самый высокий средний рейтинг так же у Центрального автономного округа.

12,5% всех заведений находятся всего на 15 улицах, самое маленькое количество заведений находится на улицах с большой удаленностью от метро и от центра города в не самых туристических местах.

**Рекомендации** <a id="recommendations"></a> 

Основываясь на референсе, данном заказчиком (кофейня "Central Park" из сериала "Друзья"), можно предположить, что видение кофейни является примерно таким: полноформатная кофейня с домашней, уютной, дружеской(семейной) атмосферой, с целевой аудиторией людей 20-35 лет, с достатком на уровне среднего класса.

Для того чтобы дать конкретные рекомендации для открытия кофейни не хватает некоторой информации, например среднего трафика по районам, сегментации кофеен(точки формата с собой/полноформатные кофейни, кофейный сегмент(спешелти/премиум), наличие кухни или своей выпечки и т.д.), для того чтобы рассмотреть их основные характеристики, позитивные и негативные аспекты.

По данным которые были рассмотрены выше и исходя из идеи заказчика пока можно выделить следующие рекомендации:

    1. Локация в шаговой доступности от метро или в месте с высоким трафиком. Возможно стоит рассмотреть как возможную локацию спальные районы, из-за более низкой конкуренции.
    2. Полноформатная кофейня с количеством посадочных мест в помещении должно быть среднее по заведениям (от 50 до 100), с возможностью разместить столы с 5-6 посадочными местами за одним столом (для компаний и семей).
    3. Наличие минимальной легкой кухни (завтраки, салаты, выпечка), что ведет к увеличению среднего чека и возможности набрать аудиторию не только за счет кофе и атмосферы, но и благодаря возможности вкусно поесть (пример Кофемании).
    4. Потенциальная возможность расширения с одной точки до небольшой сети кофеен.

**Презентация:** <https://disk.yandex.ru/i/4GtnyzDmFkhn0g>